In [1]:
# Jalankan sel ini untuk menginstal semua dependensi yang diperlukan
! pip install jupyter python-dotenv pandas minio qdrant-client langchain-openai "langchain[docarray]" luigi


  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached minio-7.2.16-py3-none-any.whl.metadata (6.5 kB)
  Using cached qdrant_client-1.15.1-py3-none-any.whl.metadata (11 kB)
  Using cached langchain_openai-0.3.30-py3-none-any.whl.metadata (2.4 kB)
  Using cached luigi-3.6.0.tar.gz (1.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached notebook-7.4.5-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached jupyterlab-4.4.6-py3-none-any.whl.metadata (16 kB)
  Using cached argon2_cffi-25.1.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using 

In [14]:
! pip install luigi==1.0.3

In [2]:

import os
import pandas as pd
from dotenv import load_dotenv
from datetime import date
from minio import Minio
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
import numpy as np
import os
import logging
import glob
import pandas as pd
from langchain_openai import OpenAIEmbeddings
import luigi
import boto3


In [77]:
load_dotenv(".env")

def get_minio_client():
    """Membuat klien Minio untuk digunakan oleh Target."""
    return Minio(
        os.getenv("MINIO_ENDPOINT"),
        access_key=os.getenv("MINIO_ACCESS_KEY"),
        secret_key=os.getenv("MINIO_SECRET_KEY"),
        secure=False
    )

def get_qdrant_client(host="localhost", port=6333):
    return QdrantClient(host=host, port=port)

def get_s3_client_for_luigi():
    """Membuat klien boto3 yang dikonfigurasi untuk MinIO."""
    return boto3.client(
        "s3",
        endpoint_url=f'http://{os.getenv("MINIO_ENDPOINT")}',
        aws_access_key_id=os.getenv("MINIO_ACCESS_KEY"),
        aws_secret_access_key=os.getenv("MINIO_SECRET_KEY")
    )

In [3]:
#@title Twitter Auth Token

twitter_auth_token = os.getenv("TWITTER_AUTH_TOKEN") # change this auth token

In [ ]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update

!sudo apt-get install nodejs -y

!node -v

[sudo] password for ori: sudo: a password is required
^C
[sudo] password for ori: 

In [17]:
# Crawl Data

filename = 'gibran2.csv'
search_keyword = 'telkomsel since:2025-08-02 until:2025-08-10 lang:id'
# Tentukan limit yang Anda inginkan
limit = 100

# Pastikan twitter_auth_token sudah didefinisikan sebelumnya, misalnya:
# twitter_auth_token = "..." 

print(f"Starting crawl for {limit} tweets...")

# Gunakan variabel {limit}, bukan {10}
! npx -y tweet-harvest@2.6.1 -o "{filename}" -s "{search_keyword}" --tab "LATEST" -l {limit} --token {twitter_auth_token}

print(f"Crawling finished. Data saved to {filename}")

Starting crawl for 100 tweets...
⠙⠹⠸⠼⠴⠦⠧⠇Tweet Harvest [v2.6.1]

Research by Helmi Satria
Use it for Educational Purposes only!

This script uses Chromium Browser to crawl data from Twitter with your Twitter auth token.
Please enter your Twitter auth token when prompted.

Note: Keep your access token secret! Don't share it with anyone else.
Note: This script only runs on your local device.


Opening twitter search page...


Filling in keywords: telkomsel since:2025-08-02 until:2025-08-10 lang:id



Your tweets saved to: /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/gibran2.csv
Total tweets saved: 18

-- Scrolling... (1)

Your tweets saved to: /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/gibran2.csv
Total tweets saved: 32


Your tweets saved to: /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/gibran2.csv
Total tweets saved: 43


Your tweets saved to: /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/gibran2.csv
Total

In [14]:
import luigi
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pathlib import Path

# Pastikan load_dotenv() dipanggil di atas
from dotenv import load_dotenv
load_dotenv(".env")

# ==============================================================================
# TUGAS 1: EKSTRAKSI TWEET (DENGAN POLA YANG BENAR)
# ==============================================================================
kemarin = date.today() - timedelta(days=1)

class ExtractTweets(luigi.Task):
    task_date = luigi.DateParameter(default=kemarin)

    def output(self):
        """
        Mendeklarasikan path akhir dari file output.
        """
        # Dapatkan direktori root proyek
        project_root = Path.cwd()
        output_dir = project_root / "tweets-data"
        
        # Pastikan direktori ada
        os.makedirs(output_dir, exist_ok=True)
        
        # Return LocalTarget dengan path absolut
        absolute_path = output_dir / f"{self.task_date.strftime('%Y-%m-%d')}.csv"
        return luigi.LocalTarget(absolute_path)

    def run(self):
        """
        Bertanggung jawab untuk menjalankan scraping dan menyimpan hasilnya
        ke path yang didefinisikan di self.output().
        """
        print(f"--- [Extract] Memulai ekstraksi data Twitter untuk {self.task_date} ---")
        
        # 1. Siapkan semua parameter yang dibutuhkan
        start_date = self.task_date.strftime('%Y-%m-%d')
        end_date_formatted = datetime.now().strftime('%Y-%m-%d')
        search_keyword = f'"telkomsel" since:{start_date} until:{end_date_formatted} lang:id'
        limit = 10
        twitter_auth_token = os.getenv('TWITTER_AUTH_TOKEN')
        
        # 2. Definisikan nama file sementara untuk hasil scraping
        temp_filename = f"temp_scrape_{self.task_date.strftime('%Y%m%d')}.csv"

        # 3. Jalankan perintah tweet-harvest untuk menyimpan ke file sementara
        command = (
            f'npx -y tweet-harvest@2.6.1 -o "{temp_filename}" '
            f'-s "{search_keyword}" --tab "LATEST" -l {limit} '
            f'--token "{twitter_auth_token}"'
        )
        print(f"Menjalankan perintah: {command}")
        os.system(command)
        project_root = Path.cwd()
        print(temp_filename)
        temp_filename=project_root/"tweets-data"/temp_filename
        print(temp_filename)
        # 4. Baca data dari file sementara dan simpan ke tujuan akhir (output)
        if os.path.exists(temp_filename):
            # Baca hasil scraping ke dalam DataFrame
            scraped_data_df = pd.read_csv(temp_filename)
            
            # Simpan DataFrame ke path akhir yang didefinisikan oleh self.output()
            # Ini sama persis dengan pola di contoh ScrapeData Anda
            scraped_data_df.to_csv(self.output().path, index=False)
            
            # 5. Hapus file sementara setelah selesai
            os.remove(temp_filename)
            print(f"--- [Extract] Selesai. Data disimpan di {self.output().path} ---")
        else:
            raise FileNotFoundError(f"Scraping gagal, file sementara tidak ditemukan: {temp_filename}")

In [15]:
luigi.build([ExtractTweets()], local_scheduler = True)

DEBUG: Checking if ExtractTweets(task_date=2025-08-19) is complete
INFO: Informed scheduler that task   ExtractTweets_2025_08_19_204f0f2397   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 45361] Worker Worker(salt=7126721917, workers=1, host=LAPTOP-GCD4EMJB, username=ori, pid=45361) running   ExtractTweets(task_date=2025-08-19)


--- [Extract] Memulai ekstraksi data Twitter untuk 2025-08-19 ---
Menjalankan perintah: npx -y tweet-harvest@2.6.1 -o "temp_scrape_20250819.csv" -s ""telkomsel" since:2025-08-19 until:2025-08-20 lang:id" --tab "LATEST" -l 10 --token "c2c3b68dd8544bd3d22cdc0e48ec05846b17f129"
Tweet Harvest [v2.6.1]

Research by Helmi Satria
Use it for Educational Purposes only!

This script uses Chromium Browser to crawl data from Twitter with your Twitter auth token.
Please enter your Twitter auth token when prompted.

Note: Keep your access token secret! Don't share it with anyone else.
Note: This script only runs on your local device.


Opening twitter search page...


-- Scrolling... (1)
Filling in keywords: telkomsel since:2025-08-19 until:2025-08-20 lang:id



Your tweets saved to: /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/temp_scrape_20250819.csv
Total tweets saved: 20


INFO: [pid 45361] Worker Worker(salt=7126721917, workers=1, host=LAPTOP-GCD4EMJB, username=ori, pid=45361) done      ExtractTweets(task_date=2025-08-19)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ExtractTweets_2025_08_19_204f0f2397   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=7126721917, workers=1, host=LAPTOP-GCD4EMJB, username=ori, pid=45361) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 ExtractTweets(task_date=2025-08-19)

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



Got 20 tweets, done scrolling...
temp_scrape_20250819.csv
/home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/temp_scrape_20250819.csv
--- [Extract] Selesai. Data disimpan di /home/ori/sentiment-project/autoUpdateData/notebooks/tweets-data/2025-08-19.csv ---


True

In [38]:
import os

# Path relatif ke folder output dari root proyek
output_path_relative = "data/raw"

# Gunakan os.getcwd() untuk mendapatkan direktori kerja saat ini (root proyek)
project_root_dir = os.getcwd()

# Gabungkan untuk mendapatkan path absolut yang andal
absolute_output_path = os.path.join(project_root_dir, output_path_relative)

print(f"Project root directory: {project_root_dir}")
print(f"Absolute output path: {absolute_output_path}")

# Sekarang Anda bisa menggunakan path absolut ini
# os.makedirs(absolute_output_path, exist_ok=True)
# if not os.path.exists(absolute_output_path):
#    ...

Project root directory: /home/ori/sentiment-project/autoUpdateData/notebooks
Absolute output path: /home/ori/sentiment-project/autoUpdateData/notebooks/data/raw
